In [1]:
import torch
from transformers import AutoModelForSequenceClassification
from tqdm import tqdm
from trl.core import LengthSampler

In [2]:
reward_model = AutoModelForSequenceClassification.from_pretrained(
        'reward_modeling_shp'
    )

In [3]:
from datasets import load_dataset
dataset_name = "stanfordnlp/SHP"
# dataset = load_dataset(dataset_name, split="train")

In [4]:
from trl import PPOConfig

config = PPOConfig(
    model_name="gpt2-medium",
    learning_rate=1.41e-5,
)

In [5]:
from transformers import AutoTokenizer

from trl import AutoModelForCausalLMWithValueHead, PPOTrainer

model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
tokenizer = AutoTokenizer.from_pretrained(config.model_name)

tokenizer.pad_token = tokenizer.eos_token

[2024-04-24 17:53:17,392] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.2
 [WARNING]  using untested triton version (2.2.0), only 1.0.0 is known to be compatible


In [6]:
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

In [7]:
with torch.no_grad():
    r = reward_model(encoded_input['input_ids'])[0].item()

In [8]:
def collator(data):
    return {key: [d[key] for d in data] for key in data[0]}

In [9]:
def build_dataset(model_name, query_dataset, input_min_text_length=2, input_max_text_length=8):
    """
    Build dataset for training. This builds the dataset from `load_dataset`, one should
    customize this function to train the model on its own dataset.

    Args:
        query_dataset (`str`):
            The name of the dataset to be loaded.

    Returns:
        dataloader (`torch.utils.data.DataLoader`):
            The dataloader for the dataset.
    """
    tokenizer = AutoTokenizer.from_pretrained(config.model_name)
    tokenizer.pad_token = tokenizer.eos_token
    # load imdb with datasets
    ds = load_dataset(query_dataset, split="train")
    ds = ds.filter(lambda x: len(x["history"]) > 200, batched=False)

    input_size = LengthSampler(input_min_text_length, input_max_text_length)

    def tokenize(sample):
        sample["input_ids"] = tokenizer.encode(sample["history"])[: input_size()]
        sample["query"] = tokenizer.decode(sample["input_ids"])
        return sample

    ds = ds.map(tokenize, batched=False, num_proc=16)
    ds.set_format(type="torch")
    return ds

In [10]:
dataset = build_dataset("gpt2-medium", dataset_name)

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

In [11]:
ppo_trainer = PPOTrainer(
    model=model,
    config=config,
    dataset=dataset,
    tokenizer=tokenizer,
    data_collator=collator,
)

In [12]:
generation_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
}

In [13]:
ppo_trainer.dataloader

In [14]:
epochs = 1
for epoch in tqdm(range(epochs), desc="epoch: "):
    for batch in tqdm(ppo_trainer.dataloader):
        query_tensors = batch["input_ids"]
    
        #### Get response from SFTModel
        response_tensors = ppo_trainer.generate(query_tensors, **generation_kwargs)
        batch["response"] = [tokenizer.decode(r.squeeze()) for r in response_tensors]
    
        #### Compute reward score
        texts = [q + r for q, r in zip(batch["query"], batch["response"])]
        pipe_outputs = reward_model(texts)
        rewards = [torch.tensor(output[1]["score"]) for output in pipe_outputs]
    
        #### Run PPO step
        stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
        ppo_trainer.log_stats(stats, batch, rewards)

# #### Save model
# ppo_trainer.save_pretrained("ppo_model/test")

epoch:   0%|          | 0/1 [00:00<?, ?it/s]

{'input_ids': [tensor([ 40, 836], device='cuda:0'), tensor([   58,  8777,   286, 20902], device='cuda:0'), tensor([2437,  857], device='cuda:0'), tensor([2437,  466], device='cuda:0'), tensor([   58,  9655,    52,    60,  4231, 41008,  1547], device='cuda:0'), tensor([ 5211, 27170,   508,  1816,   284, 17168,    11], device='cuda:0'), tensor([ 2061, 32139,   466,   345,   892,   389,  2861], device='cuda:0'), tensor([2437,  890], device='cuda:0'), tensor([ 2061,  1729,    12, 36380,  3946,   466], device='cuda:0'), tensor([7149, 4213], device='cuda:0'), tensor([   14,    81,    14,  2093, 28864,   418, 16982], device='cuda:0'), tensor([  33,  868, 4899], device='cuda:0'), tensor([5840, 1972, 6405], device='cuda:0'), tensor([2061,  318,  351, 2506, 8868, 7543], device='cuda:0'), tensor([3792, 2055, 4152,  922, 1576,  284], device='cuda:0'), tensor([2061,  481, 8494], device='cuda:0'), tensor([2061,  447,  247,   82,  257, 3621], device='cuda:0'), tensor([  37, 5631, 1139], device='cuda:

{'input_ids': [tensor([   58, 12805,    60,  1680,   314,  3512], device='cuda:0'), tensor([   40,  1842, 16871,    11], device='cuda:0'), tensor([24187,    53,    25,  9842,  1112], device='cuda:0'), tensor([24187,    53,    25, 11204,   925,   284], device='cuda:0'), tensor([42969, 12165,   502,   503,   379], device='cuda:0'), tensor([2061,  389,  534], device='cuda:0'), tensor([ 6090,   314, 48612], device='cuda:0'), tensor([3792,  340], device='cuda:0'), tensor([ 2061,   318,   287,   616, 10601,    30], device='cuda:0'), tensor([24187,    53,    25,   383,  6224,   290], device='cuda:0'), tensor([6090, 2130,  508,  447,  247,   82, 2089], device='cuda:0'), tensor([   58,  9697, 15650,    60], device='cuda:0'), tensor([  58, 4146], device='cuda:0'), tensor([ 1639,   821, 13640,   319,  1194,  5440,  1231], device='cuda:0'), tensor([25214, 26959, 38268,  7171,    25,   314], device='cuda:0'), tensor([2437, 1884,  318,  340,  329,  281, 8705], device='cuda:0'), tensor([   32,  3043,

{'input_ids': [tensor([18673,   418, 16982, 26353,   290], device='cuda:0'), tensor([2061,  389], device='cuda:0'), tensor([    7,  6242,    11,  3340,   737, 22505, 32289], device='cuda:0'), tensor([30821, 33072], device='cuda:0'), tensor([11215,    85,    25,   551], device='cuda:0'), tensor([  58, 8141,   60, 2011, 6478, 1297], device='cuda:0'), tensor([3546,  852, 4137,  284, 1445, 1909, 2233], device='cuda:0'), tensor([2061, 4583,  318,  564,  250], device='cuda:0'), tensor([4366, 7650], device='cuda:0'), tensor([16977,   357], device='cuda:0'), tensor([  40, 1392,  257,  362], device='cuda:0'), tensor([17184,  8942], device='cuda:0'), tensor([ 2061,  1430,   466, 12037], device='cuda:0'), tensor([28620,   508,   423,  3111, 10522], device='cuda:0'), tensor([46445,  2865,  7622, 16568,  1176,   329,  5186], device='cuda:0'), tensor([   7, 5673,    8, 2011, 1545,  373], device='cuda:0'), tensor([ 5962,   640,  1642, 32051,   422, 11945], device='cuda:0'), tensor([ 8241,  2073, 1040

{'input_ids': [tensor([20827,  6607], device='cuda:0'), tensor([24187,    53,    25, 19940,   278,  7971], device='cuda:0'), tensor([3666, 3290,  468, 1278,  559,  785,   64], device='cuda:0'), tensor([5195,  466], device='cuda:0'), tensor([29252,  1659, 33263, 24796,  2409, 33511], device='cuda:0'), tensor([18438,   391,   588,   314], device='cuda:0'), tensor([3666, 9998, 5115], device='cuda:0'), tensor([   40,  1392,   257,   362, 23160], device='cuda:0'), tensor([ 2725,    35, 16392,  1762,  2354], device='cuda:0'), tensor([13921,  2011, 21494,  6075,   360,     6], device='cuda:0'), tensor([5840,  314, 4854,   14], device='cuda:0'), tensor([ 6090,   345, 28450], device='cuda:0'), tensor([18438,   391,   588,   314,  1101], device='cuda:0'), tensor([   58, 12218], device='cuda:0'), tensor([  37, 6255, 2563], device='cuda:0'), tensor([ 2061,   338,   262, 38889,  1693,   281], device='cuda:0'), tensor([5195,  389,  612, 2048,  645, 6918], device='cuda:0'), tensor([2437,  466], devic

...

{'input_ids': [tensor([18438,   391,   588,   314], device='cuda:0'), tensor([ 5211,   484,  6330, 50046,   287,   299, 31469], device='cuda:0'), tensor([13921,  2687], device='cuda:0'), tensor([  54,  901, 3724], device='cuda:0'), tensor([1532,  616], device='cuda:0'), tensor([ 6090,   314, 16611,  8246], device='cuda:0'), tensor([3666, 6621, 3888,  287,  351,  502,  718], device='cuda:0'), tensor([   58, 12805,    60], device='cuda:0'), tensor([24187,    53,    25], device='cuda:0'), tensor([ 464, 2368, 5079, 1220], device='cuda:0'), tensor([   58, 12218,  8842,    60], device='cuda:0'), tensor([ 2061, 10512,   318,  5836,   287,   534,  5011], device='cuda:0'), tensor([19242,   612,   587,   257,  4302], device='cuda:0'), tensor([ 3666,  3656, 37264,   319], device='cuda:0'), tensor([  58, 8248, 6176], device='cuda:0'), tensor([   40,   423,   655,   587, 22527,   546], device='cuda:0'), tensor([26788,   373, 14716, 27163,   656,   616], device='cuda:0'), tensor([  32, 1545], device

{'input_ids': [tensor([  58, 3697,   60, 4222], device='cuda:0'), tensor([5211,  345,  892, 2267, 2482], device='cuda:0'), tensor([24187,    53,    25,  1081,  1052,  5510], device='cuda:0'), tensor([ 2061,   534, 39877], device='cuda:0'), tensor([  35, 2178, 1808,  546, 1097, 8705,  532], device='cuda:0'), tensor([2061,  338,  534, 4459,  319, 4445, 5059], device='cuda:0'), tensor([ 3109,    12, 48912,   318,  3501,  1200], device='cuda:0'), tensor([  818,   644,   835,   857, 40258], device='cuda:0'), tensor([13856,  6243,  6920,    12], device='cuda:0'), tensor([38864,    60,   314], device='cuda:0'), tensor([18438,   391,   588,   314,  1101,  1936], device='cuda:0'), tensor([   58, 41074,    60,  2080,    67,  1809], device='cuda:0'), tensor([   40,  2904, 18303,   287, 19663,  8705], device='cuda:0'), tensor([14282,  9749,  7622], device='cuda:0'), tensor([1135, 7213, 7586], device='cuda:0'), tensor([   40,  5839, 14996], device='cuda:0'), tensor([   58, 13195, 17980,  2319], dev

epoch:   0%|          | 0/1 [00:33<?, ?it/s]

{'input_ids': [tensor([   58, 29551,    60,   314], device='cuda:0'), tensor([ 9509,  8143, 12037,   644,   389,   617], device='cuda:0'), tensor([    1, 22058], device='cuda:0'), tensor([ 2437,   284,  2652, 13347,   981, 16871,    30], device='cuda:0'), tensor([ 3666, 17834, 25949], device='cuda:0'), tensor([11633,   345,  2555,   284,  7715,   706,  9581], device='cuda:0'), tensor([ 3666,   662,  6036,  4957,   338, 46601,   318], device='cuda:0'), tensor([18438,   391,   588,   314,  1101], device='cuda:0'), tensor([   58, 18308, 14179,    60,   314,  1043,   257], device='cuda:0'), tensor([ 34, 377], device='cuda:0'), tensor([13014,  1517,   345,  1683, 22979,    30,   314], device='cuda:0'), tensor([28937,   278,  2057,   329,   661], device='cuda:0'), tensor([24187,    53,    25,   383,  1109], device='cuda:0'), tensor([9492,   69, 1586], device='cuda:0'), tensor([2437,  284], device='cuda:0'), tensor([24187,    53,    25,  4042,  4380], device='cuda:0'), tensor([24187,    53,  

KeyboardInterrupt: 